*Proyecto base y set de datos originales extraidos de <a src="https://datascience4business.com/01_pdsm-tps-optin">datascience4business</a>*

___

## Aplicando Machine Learning para predecir abandonos

Una vez analizados y procesados los datos originales (`Abandono_EDA.ipynb`), aplicaremos modelos de machine learning (aprendizaje maquina) a los datos de la empresa "Y" para poder predecir con mas exactitud las repercuciones del abandono de empleados.

### Importamos Librerias

In [85]:
import numpy as np
import pandas as pd
from eda_vx import Eda
from ml_vx import Tools, ML

### Cargamos los datos ya modificados

In [86]:
df_ml = pd.read_csv('abandono_modificado.csv', sep = ',')
df_ml

,edad,abandono,viajes,departamento,distancia_casa,educacion,carrera,satisfaccion_entorno,implicacion,nivel_laboral,...,evaluacion,satisfaccion_companeros,nivel_acciones,anos_experiencia,num_formaciones_ult_ano,anos_compania,anos_desde_ult_promocion,anos_con_manager_actual,salario_anual,impacto_economico
0,41,Yes,Travel_Rarely,Sales,1,Universitaria,Life Sciences,Media,Alta,2,...,Alta,Baja,0,8,0,6,0,5,71916,14670.864
1,49,No,Travel_Frequently,Research & Development,8,Secundaria,Life Sciences,Alta,Media,2,...,Muy_Alta,Muy_Alta,1,10,3,10,1,7,61560,12558.240
2,37,Yes,Travel_Rarely,Research & Development,2,Secundaria,Other,Muy_Alta,Media,1,...,Alta,Media,0,7,3,0,0,0,25080,4037.880
3,33,No,Travel_Frequently,Research & Development,3,Universitaria,Life Sciences,Muy_Alta,Alta,1,...,Alta,Alta,0,8,3,8,3,0,34908,6876.876
4,27,No,Travel_Rarely,Research & Development,2,Universitaria,Medical,Baja,Alta,1,...,Alta,Muy_Alta,1,6,3,2,2,2,41616,8198.352
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,No,Travel_Frequently,Research & Development,23,Master,Medical,Alta,Muy_Alta,2,...,Alta,Alta,1,17,3,5,0,3,30852,6077.844
1466,39,No,Travel_Rarely,Research & Development,6,Secundaria,Medical,Muy_Alta,Media,3,...,Alta,Baja,1,9,5,7,1,7,119892,25177.320
1467,27,No,Travel_Rarely,Research & Development,4,Master,Life Sciences,Media,Muy_Alta,2,...,Muy_Alta,Media,1,6,0,6,0,3,73704,15035.616
1468,49,No,Travel_Frequently,Sales,2,Secundaria,Medical,Muy_Alta,Media,2,...,Alta,Muy_Alta,0,17,3,9,0,8,64680,13194.720


1) Transformamos las variables categoricas a numericas mediante el metodo "one hot encode"

In [87]:
# Tambien se podria hacer con los metodos "dummy" (pd.Dummies) o con "label" (LabelEncoder)
df_ml = Eda.convertir_a_numericas(df_ml, target="abandono", metodo="ohe", drop_first=True)
df_ml

,edad,distancia_casa,nivel_laboral,salario_mes,num_empresas_anteriores,incremento_salario_porc,nivel_acciones,anos_experiencia,num_formaciones_ult_ano,anos_compania,...,satisfaccion_trabajo_Media,satisfaccion_trabajo_Muy_Alta,estado_civil_Married,estado_civil_Single,horas_extra_Yes,evaluacion_Muy_Alta,satisfaccion_companeros_Baja,satisfaccion_companeros_Media,satisfaccion_companeros_Muy_Alta,abandono
0,41,1,2,5993,8,11,0,8,0,6,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1
1,49,8,2,5130,1,23,1,10,3,10,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0
2,37,2,1,2090,6,15,0,7,3,0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1
3,33,3,1,2909,1,11,0,8,3,8,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0
4,27,2,1,3468,9,12,1,6,3,2,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,23,2,2571,4,17,1,17,3,5,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1466,39,6,3,9991,4,15,1,9,5,7,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0
1467,27,4,2,6142,1,20,1,6,0,6,...,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0
1468,49,2,2,5390,2,14,0,17,3,9,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0


2) Estandarizamos las variables numericas para nivelar sus pesos y mezclamos

In [88]:
# Aplicamos la estandarización utilizando el Z-Score (media 0, desviación estándar 1)
df_ml = Eda.estandarizar_variables(df_ml, target="abandono", metodo="zscore")
df_ml

,edad,distancia_casa,nivel_laboral,salario_mes,num_empresas_anteriores,incremento_salario_porc,nivel_acciones,anos_experiencia,num_formaciones_ult_ano,anos_compania,...,satisfaccion_trabajo_Media,satisfaccion_trabajo_Muy_Alta,estado_civil_Married,estado_civil_Single,horas_extra_Yes,evaluacion_Muy_Alta,satisfaccion_companeros_Baja,satisfaccion_companeros_Media,satisfaccion_companeros_Muy_Alta,abandono
0,0.446350,-1.010909,-0.057788,-0.108350,2.125136,-1.150554,-0.932014,-0.421642,-2.171982,-0.164613,...,-0.563209,-0.319295,-0.918921,1.458650,1.591746,-0.426230,2.079925,-0.509549,-0.645124,1
1,1.322365,-0.147150,-0.057788,-0.291719,-0.678049,2.129306,0.241988,-0.164511,0.155707,0.488508,...,1.775540,-0.319295,1.088232,-0.685565,-0.628241,2.346151,-0.480787,-0.509549,1.550090,0
2,0.008343,-0.887515,-0.961486,-0.937654,1.324226,-0.057267,-0.932014,-0.550208,0.155707,-1.144294,...,1.775540,-0.319295,-0.918921,1.458650,1.591746,-0.426230,-0.480787,1.962520,-0.645124,1
3,-0.429664,-0.764121,-0.961486,-0.763634,-0.678049,-1.150554,-0.932014,-0.421642,0.155707,0.161947,...,-0.563209,-0.319295,1.088232,-0.685565,1.591746,-0.426230,-0.480787,-0.509549,-0.645124,0
4,-1.086676,-0.887515,-0.961486,-0.644858,2.525591,-0.877232,0.241988,-0.678774,0.155707,-0.817734,...,-0.563209,-0.319295,1.088232,-0.685565,-0.628241,-0.426230,-0.480787,-0.509549,1.550090,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,-0.101159,1.703764,-0.057788,-0.835451,0.523316,0.489376,0.241988,0.735447,0.155707,-0.327893,...,-0.563209,3.131904,1.088232,-0.685565,-0.628241,-0.426230,-0.480787,-0.509549,-0.645124,0
1466,0.227347,-0.393938,0.845911,0.741140,0.523316,-0.057267,0.241988,-0.293077,1.707500,-0.001333,...,1.775540,-0.319295,1.088232,-0.685565,-0.628241,-0.426230,2.079925,-0.509549,-0.645124,0
1467,-1.086676,-0.640727,-0.057788,-0.076690,-0.678049,1.309341,0.241988,-0.678774,-2.171982,-0.164613,...,-0.563209,3.131904,1.088232,-0.685565,1.591746,2.346151,-0.480787,1.962520,-0.645124,0
1468,1.322365,-0.887515,-0.057788,-0.236474,-0.277594,-0.330589,-0.932014,0.735447,0.155707,0.325228,...,1.775540,-0.319295,1.088232,-0.685565,-0.628241,-0.426230,-0.480787,-0.509549,1.550090,0


3) Analizaremos el peso de las variables para predecir el abandono

In [89]:
Tools.importancia_variables(df_ml, target="abandono", tipo_problema="clasificacion", random_state=1)


 ACCURACY: 0.8564625850340135


(None,
                                Variable  Importancia
 0                         salario_anual     0.074595
 1                           salario_mes     0.066706
 2                     impacto_economico     0.066431
 3                                  edad     0.058962
 4                       horas_extra_Yes     0.053297
 5                        distancia_casa     0.052724
 6                      anos_experiencia     0.052658
 7                         anos_compania     0.045837
 8               incremento_salario_porc     0.041017
 9               num_empresas_anteriores     0.038979
 10              anos_con_manager_actual     0.036349
 11             anos_desde_ult_promocion     0.032475
 12              num_formaciones_ult_ano     0.031207
 13                       nivel_acciones     0.029511
 14            satisfaccion_entorno_Baja     0.019861
 15                        nivel_laboral     0.018631
 16                  estado_civil_Single     0.018179
 17             viaje

Utilizamos un modelo de **Random Forest** (Bosque Aleatorio) para analizar los pesos, vemos que el scoring que utilizamos (Accuracy) nos esta dando 1.0 lo que podria estar indicando colinealidad en los datos. Si recordamos, creamos la variable `salario_anual` que es una convinacion de otras variables. Intentemos entonces eliminar esta variable y volver a correr el modelo

In [90]:
df_ml.drop('impacto_economico',axis=1,inplace=True)

Volvemos a correr el modelo

In [91]:
Tools.importancia_variables(df_ml, target="abandono",tipo_problema="clasificacion", random_state=1)


 ACCURACY: 0.8544217687074831


(None,
                                Variable  Importancia
 0                         salario_anual     0.081607
 1                           salario_mes     0.076868
 2                                  edad     0.069308
 3                      anos_experiencia     0.060308
 4                        distancia_casa     0.059201
 5                       horas_extra_Yes     0.057241
 6                         anos_compania     0.050047
 7               incremento_salario_porc     0.043814
 8               num_empresas_anteriores     0.038073
 9               anos_con_manager_actual     0.035315
 10              num_formaciones_ult_ano     0.033201
 11             anos_desde_ult_promocion     0.033194
 12                       nivel_acciones     0.029639
 13                        nivel_laboral     0.024053
 14            satisfaccion_entorno_Baja     0.019065
 15                  estado_civil_Single     0.018315
 16             viajes_Travel_Frequently     0.016656
 17         satisfacc

Vemos que se corrijio el error, ahora si podemos analizar el resultado y lo primero que vemos es que hay variables cuyo peso en la prediccion es muy baja (`puesto_Manager` y `puesto_Research Director`) con menos del %0.3. Aqui tenemos dos caminos, dejarlas o eliminarlas. Tomemos el segundo camino y quitemos estas variables, para eso usarameos el mismo metodo utilizado pero cambiando algunos parametros ...(..., *eliminar=True, umbral=0.003*) y agregando un [1] porque nos interesa guardar solo el dataframe y no el score

In [92]:
df_ml = Tools.importancia_variables(df_ml, target="abandono", tipo_problema="clasificacion", eliminar=True, umbral=0.003, random_state=1)[1]


 ACCURACY: 0.8544217687074831


Comprobamos si se eliminaron correctamente

In [93]:
try:
    df_ml["puesto_Manager"]
except Exception:
    print("Correctamente eliminado 'puesto_Manager'")
try:
    df_ml["puesto_Research Director"]
except Exception:
    print("Correctamente eliminado 'puesto_Research Director'")

Correctamente eliminado 'puesto_Manager'
Correctamente eliminado 'puesto_Research Director'


4) Crearemos cuatro Modelos y analizaremos cual se ajusta mejor a los datos; Podemos usar una busqueda de los mejores hiperparametros utilizando el parametro *``rejilla=True``* pero como puede ser mucho mas lento lo dejaremos para la proxima seccion. Por ultimo, graficaremos una matriz de confusion con los resultados

* 4.1. Modelo **AdaBoost**

In [94]:
ML.modelo_adaboost(df=df_ml, target="abandono", tipo_problema="clasificacion", graficar=True, random_state=1)

,Metrica,Valor,Explicacion
0,Accuracy,0.833333,(Exactitud) Proporción de muestras correctamente clasificadas.
1,Precision,0.846154,Proporción de muestras positivas correctamente identificadas entre todas las muestras clasificadas como positivas.
2,Recall,0.189655,(Sensibilidad) Proporción de muestras positivas correctamente identificadas entre todas las muestras reales positivas.
3,F1-score,0.309859,Media armónica entre precisión y recall. Es útil cuando hay un desequilibrio entre las clases.
4,AUC-ROC,0.590590,"Área bajo la curva ROC, que mide la capacidad de discriminación del modelo."


* 4.2. Modelo **CatBoost**

In [118]:
ML.modelo_catboost(df=df_ml, target="abandono", tipo_problema="clasificacion", graficar=True, random_state=1, save_model=True)

0:	learn: 0.6299832	test: 0.6372881	best: 0.6372881 (0)	total: 2.86ms	remaining: 283ms
50:	learn: 0.2915769	test: 0.3881610	best: 0.3880962 (49)	total: 100ms	remaining: 96.4ms
Stopped by overfitting detector  (10 iterations wait)

bestTest = 0.3824935817
bestIteration = 77

Shrink model to first 78 iterations.


,Metrica,Valor,Explicacion
0,Accuracy,0.843537,(Exactitud) Proporción de muestras correctamente clasificadas.
1,Precision,0.833333,Proporción de muestras positivas correctamente identificadas entre todas las muestras clasificadas como positivas.
2,Recall,0.258621,(Sensibilidad) Proporción de muestras positivas correctamente identificadas entre todas las muestras reales positivas.
3,F1-score,0.394737,Media armónica entre precisión y recall. Es útil cuando hay un desequilibrio entre las clases.
4,AUC-ROC,0.622954,"Área bajo la curva ROC, que mide la capacidad de discriminación del modelo."


* 4.3. Modelo **XgBoost**

In [96]:
ML.modelo_xgboost(df=df_ml, target="abandono", tipo_problema="clasificacion", graficar=True, random_state=1)

,Metrica,Valor,Explicacion
0,Accuracy,0.833333,(Exactitud) Proporción de muestras correctamente clasificadas.
1,Precision,0.714286,Proporción de muestras positivas correctamente identificadas entre todas las muestras clasificadas como positivas.
2,Recall,0.258621,(Sensibilidad) Proporción de muestras positivas correctamente identificadas entre todas las muestras reales positivas.
3,F1-score,0.379747,Media armónica entre precisión y recall. Es útil cuando hay un desequilibrio entre las clases.
4,AUC-ROC,0.616598,"Área bajo la curva ROC, que mide la capacidad de discriminación del modelo."


* 4.4. Modelo **LightGBM**

In [97]:
ML.modelo_lightgbm(df=df_ml, target="abandono", tipo_problema="clasificacion", graficar=True, random_state=1)

[LightGBM] [Warning] num_iterations is set=100, num_boost_round=100 will be ignored. Current value: num_iterations=100


,Metrica,Valor,Explicacion
0,Accuracy,0.833333,(Exactitud) Proporción de muestras correctamente clasificadas.
1,Precision,0.800000,Proporción de muestras positivas correctamente identificadas entre todas las muestras clasificadas como positivas.
2,Recall,0.206897,(Sensibilidad) Proporción de muestras positivas correctamente identificadas entre todas las muestras reales positivas.
3,F1-score,0.328767,Media armónica entre precisión y recall. Es útil cuando hay un desequilibrio entre las clases.
4,AUC-ROC,0.597092,"Área bajo la curva ROC, que mide la capacidad de discriminación del modelo."


## Mejorado el Modelo seleccionado

Basado en los resultados (que pueden variar de un entrenamiento si no usamos una semilla unica como en nuestro caso con *random_state=1*), seleccionaremos el que tenga una mejor **accuracy**, en este caso el modelo ``CatBoost`` es quien genero el mejor scoring con un 0.8435. Ahora procuraremos mejorar la eficiencia de este modelo agregando una busqueda de hiperparametros, si no lo logramos mejorar nos quedaremos con el primero que entrenamos.

5) Buscamos mejor el modelo buscando hiperparametros mas eficientes (*rejilla=True*) y guardamos el modelo entrenado (*save_model=True*)

In [98]:
ML.modelo_catboost(df_ml, "abandono", "clasificacion", random_state=1, save_model=True, rejilla=True)

0:	learn: 0.6861972	total: 2.64ms	remaining: 262ms
1:	learn: 0.6792170	total: 5.94ms	remaining: 291ms
2:	learn: 0.6723617	total: 8.73ms	remaining: 282ms
3:	learn: 0.6659589	total: 12.2ms	remaining: 293ms
4:	learn: 0.6583291	total: 15.2ms	remaining: 289ms
5:	learn: 0.6523735	total: 17.9ms	remaining: 280ms
6:	learn: 0.6460317	total: 21ms	remaining: 279ms
7:	learn: 0.6395796	total: 23.5ms	remaining: 271ms
8:	learn: 0.6327993	total: 26.4ms	remaining: 266ms
9:	learn: 0.6264795	total: 29ms	remaining: 261ms
10:	learn: 0.6209586	total: 31.5ms	remaining: 255ms
11:	learn: 0.6151641	total: 34.2ms	remaining: 251ms
12:	learn: 0.6095085	total: 37.3ms	remaining: 250ms
13:	learn: 0.6023433	total: 39.8ms	remaining: 244ms
14:	learn: 0.5962966	total: 42.2ms	remaining: 239ms
15:	learn: 0.5911031	total: 44.5ms	remaining: 234ms
16:	learn: 0.5848340	total: 46.9ms	remaining: 229ms
17:	learn: 0.5797562	total: 49.3ms	remaining: 225ms
18:	learn: 0.5737677	total: 52ms	remaining: 222ms
19:	learn: 0.5676498	total: 

,Metrica,Valor,Explicacion
0,Accuracy,0.829932,(Exactitud) Proporción de muestras correctamente clasificadas.
1,Precision,0.833333,Proporción de muestras positivas correctamente identificadas entre todas las muestras clasificadas como positivas.
2,Recall,0.172414,(Sensibilidad) Proporción de muestras positivas correctamente identificadas entre todas las muestras reales positivas.
3,F1-score,0.285714,Media armónica entre precisión y recall. Es útil cuando hay un desequilibrio entre las clases.
4,AUC-ROC,0.581970,"Área bajo la curva ROC, que mide la capacidad de discriminación del modelo."


Como el modelo nos devuelve un accuracy de 0.83, nos quedaremos con el modelo entrenado anteriormente

6) Cargamos el modelo guardado

In [119]:
import joblib


# Ruta y nombre del archivo donde se guardó el modelo
model_filename = "catboost.pkl"

# Cargar el modelo
loaded_model = joblib.load(model_filename)

# Dividimos los datos para generar la prediccion
X = df_ml.drop("abandono",axis=1)

# Generamos las predicciones
y_pred = loaded_model.predict_proba(X).T[1]
y_pred.shape

(1470,)

5) Agregamos las predicciones al dataset original (preprocesado)

In [129]:
# Cargamos el dataset modificado y agregando una columna con el id de cada empleado
id = pd.read_csv('id_empleado.csv')
df = pd.read_csv('abandono_modificado.csv', sep = ',')
df["id_empleado"] = id

# Le agregamos las predicciones
df["predicciones"] = y_pred
df

,edad,abandono,viajes,departamento,distancia_casa,educacion,carrera,satisfaccion_entorno,implicacion,nivel_laboral,...,nivel_acciones,anos_experiencia,num_formaciones_ult_ano,anos_compania,anos_desde_ult_promocion,anos_con_manager_actual,salario_anual,impacto_economico,id_empleado,predicciones
0,41,Yes,Travel_Rarely,Sales,1,Universitaria,Life Sciences,Media,Alta,2,...,0,8,0,6,0,5,71916,14670.864,6016,0.407775
1,49,No,Travel_Frequently,Research & Development,8,Secundaria,Life Sciences,Alta,Media,2,...,1,10,3,10,1,7,61560,12558.240,9662,0.029999
2,37,Yes,Travel_Rarely,Research & Development,2,Secundaria,Other,Muy_Alta,Media,1,...,0,7,3,0,0,0,25080,4037.880,2896,0.632462
3,33,No,Travel_Frequently,Research & Development,3,Universitaria,Life Sciences,Muy_Alta,Alta,1,...,0,8,3,8,3,0,34908,6876.876,1513,0.366300
4,27,No,Travel_Rarely,Research & Development,2,Universitaria,Medical,Baja,Alta,1,...,1,6,3,2,2,2,41616,8198.352,6580,0.165367
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,No,Travel_Frequently,Research & Development,23,Master,Medical,Alta,Muy_Alta,2,...,1,17,3,5,0,3,30852,6077.844,3017,0.057219
1466,39,No,Travel_Rarely,Research & Development,6,Secundaria,Medical,Muy_Alta,Media,3,...,1,9,5,7,1,7,119892,25177.320,2890,0.018181
1467,27,No,Travel_Rarely,Research & Development,4,Master,Life Sciences,Media,Muy_Alta,2,...,1,6,0,6,0,3,73704,15035.616,1495,0.100470
1468,49,No,Travel_Frequently,Sales,2,Secundaria,Medical,Muy_Alta,Media,2,...,0,17,3,9,0,8,64680,13194.720,3570,0.065710


## Exploramos los resultados

Analizamos la cantidad de empleados, segun el area de trabajo, con riesgo de abandonar la empresa (utilizaremos un umbral de 50% o mas) y su impacto economico

In [130]:
# Filtrar las observaciones
filtro = (df["abandono"] == "No") & (df["predicciones"] > 0.5)

# Realizar el groupby, contar las observaciones y calcular la suma redondeada de "impacto_economico"
result = df[filtro].groupby("departamento").agg(conteo=pd.NamedAgg(column='abandono', aggfunc='size'),
                                                 suma_impacto=pd.NamedAgg(column='impacto_economico', \
                                                    aggfunc=lambda x: round(x.sum(), 0)),
                                                 id_empleado=pd.NamedAgg(column='id_empleado', \
                                                   aggfunc=lambda x: x.tolist())).reset_index()

result


,departamento,conteo,suma_impacto,id_empleado
0,Research & Development,4,16709.0,"[9502, 3948, 7448, 9525]"
1,Sales,2,6917.0,"[2967, 5320]"


In [131]:
print(f'El Costo Total sera de u$s {result.suma_impacto.sum()}')

El Costo Total sera de u$s 23626.0


Conclusion:
* Con un 50% de probabilidad, perderemos seis empleados el proximo año, dos en ventas y otros cuatro en Research & Development
* El costo asociado a ambas renuncias sera de unos u$s 23,626.0
* En la columna id_empleado podemos ver el numero de identificacion de los empleados en riesgo de renuncia